In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from ordercompute import orderCompute

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import backtrader as bt
import pyfolio as pf
%matplotlib inline

In [ ]:
class someStrat(bt.Strategy):
    
    params = (
        ('p1', 10),
        ('p2', 6),
    )
    
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s %s, %s' % (dt.isoformat(), self.msP.time, txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        #self.dataopen = self.datas[0].open
        self.datalow = self.datas[0].low
        self.datahigh = self.datas[0].high
        
        self.fromdate = datetime.datetime(2016, 3, 2) #
        self.order = None

        self.longtarget = 100 #
        self.longstop = 100 #
        self.shorttarget = 100 #
        self.shortstop = 100 #

        self.opendate = self.data.datetime.date() #
        self.ordertype = None #

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            self.bar_executed = len(self)

        # Write down: no pending order
        self.order = None
        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f\n' %
                 (trade.pnl, trade.pnlcomm))
        
    def next(self):
        self.msP.time = self.data.datetime.time().isoformat()
        self.msP.date = self.data.datetime.date().isoformat()

        # Check if an order is pending
        if self.order:
            return
        
        # Check if we are in the market
        if (not self.position and <condition>):
                
                # > Get OHLC, (self.dataopen[0])
                if <open_conditions>:
                    self.longtarget = None #Set
                    self.longstop = None # Set
                    self.opendate = self.data.datetime.date()
                    self.order = self.buy()
                    self.log('LONG CREATE, P:%.4f, T:%.4f, SL:%.4f' % (self.dataclose[0], <target>, <stop>))
                    self.ordertype = 'Long'

                if <open_conditions>:
                    self.shorttarget = None # Set
                    self.shortstop = None # Set
                    self.opendate = self.data.datetime.date()
                    self.order = self.sell()
                    self.log('SHORT CREATE, P:%.4f, T:%.4f, SL:%.4f' % (self.dataclose[0], <target>, <stop>))
                    self.ordertype = 'Short'

        else:
            if self.ordertype == 'Long': # and not order.isbuy():
                if self.dataclose[0] >= self.longtarget or self.dataclose[0] <= self.longstop: #or 
                    self.log('LONG POSITION CLOSE, %.4f' % self.dataclose[0])
                    self.order = self.close()
                    self.ordertype = None
            elif self.ordertype == 'Short': #and not self.order.issell():
                if self.dataclose[0] <= self.shorttarget or self.dataclose[0] >= self.shortstop:
                    self.log('SHORT POSITION CLOSE, %.4f' % self.dataclose[0])
                    self.order = self.close()
                    self.ordertype = None
        
    # For optimization only
    '''
    def stop(self):
        self.log('Ending Value %.2f' %
                 self.broker.getvalue(), doprint=True)
    '''

In [ ]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # Single backtest
    cerebro.addstrategy(SRStrategy)
    
    # Optimization
    '''
    cerebro.optstrategy(
        SRStrategy,
        target_q=range(5, 12),
        stop_q=range(2, 7),
    )
    '''
    # GitHub/stratrepo/ ->
    datapath = 'data/tickdump/EURUSD_20160301-20170301.csv' #YHOO1617.csv'
    
    # Create a Data Feed
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        fromdate=datetime.datetime(2016, 3, 2),
        todate=datetime.datetime(2016, 3, 20),
        timeframe=bt.TimeFrame.Minutes,
        dtformat='%d.%m.%Y %H:%M:%S.000')

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)
    print('Starting Portfolio Value: %.4f' % cerebro.broker.getvalue())
    
    cerebro.addsizer(bt.sizers.FixedSize, stake=90000)
    cerebro.broker.setcommission(commission=0.0)
    cerebro.addanalyzer(bt.analyzers.PyFolio)
    
    # Run over everything
    results = cerebro.run()
    #cerebro.plot()
    
    # Print out the final net value
    print('Final Portfolio Value: %.4f' % cerebro.broker.getvalue())
    # Pyfolio integration, gross_lev not functioning properly
    strat = results[0]
    pyfoliozer = strat.analyzers.getbyname('pyfolio')
    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    
    pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    #gross_lev=1000,#gross_lev,
    live_start_date='2016-03-17',  # This date is sample specific
    round_trips=True)
